In [1]:
import pandas as pd
import numpy as np
import random
import sys
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeaveOneOut, cross_val_score
import numpy as np
from sklearn import svm
from sklearn.svm import SVC
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


## 1. Dataset Calling


In [2]:
class DatasetHelper:
    
    def __init__(self):
        df = pd.read_csv('datasetPF.csv')
        df = df[['project',
                'actual_effort',
                'size_ucp',
                'F1', 'F2','F3','F4','F5','F6','F7','F8']]
        self.datasets = df

    # return all dataset
    def all_datasets(self):
        data_array = self.datasets.to_dict(orient='index')

        return data_array

    # return all dataset - 1
    # leave one dataset from index
    def dataset_leave_one(self, index):
        leave_one = self.datasets.drop(index)
        leave_one = leave_one.reset_index(drop=True)
        leave_one = leave_one.to_dict(orient='index')
        return leave_one

In [3]:
# Test
loo = DatasetHelper()
loo.dataset_leave_one(119)

{0: {'project': 0,
  'actual_effort': 2124,
  'size_ucp': 118,
  'F1': 2,
  'F2': 2,
  'F3': 2,
  'F4': 2,
  'F5': 2,
  'F6': 3,
  'F7': 4,
  'F8': 0},
 1: {'project': 1,
  'actual_effort': 1430,
  'size_ucp': 130,
  'F1': 4,
  'F2': 3,
  'F3': 3,
  'F4': 4,
  'F5': 4,
  'F6': 3,
  'F7': 3,
  'F8': 0},
 2: {'project': 2,
  'actual_effort': 1445,
  'size_ucp': 85,
  'F1': 3,
  'F2': 3,
  'F3': 3,
  'F4': 3,
  'F5': 3,
  'F6': 4,
  'F7': 3,
  'F8': 0},
 3: {'project': 3,
  'actual_effort': 4895,
  'size_ucp': 275,
  'F1': 4,
  'F2': 4,
  'F3': 4,
  'F4': 5,
  'F5': 4,
  'F6': 4,
  'F7': 2,
  'F8': 0},
 4: {'project': 4,
  'actual_effort': 2420,
  'size_ucp': 110,
  'F1': 3,
  'F2': 2,
  'F3': 2,
  'F4': 3,
  'F5': 3,
  'F6': 3,
  'F7': 4,
  'F8': 0},
 5: {'project': 5,
  'actual_effort': 2080,
  'size_ucp': 65,
  'F1': 2,
  'F2': 2,
  'F3': 2,
  'F4': 2,
  'F5': 2,
  'F6': 3,
  'F7': 3,
  'F8': 4},
 6: {'project': 6,
  'actual_effort': 1265,
  'size_ucp': 55,
  'F1': 3,
  'F2': 3,
  'F3'

## 2. Bisecting K-Medoid

In [4]:
EXPONENT = 2

class BisectingKMedoid:
    
    def __init__(self, datasets):
        self.datasets = datasets
        self.clusters = []

    def random_medoid_tuple(self, arr_dataset):
        if (len(arr_dataset) == 1):
            return arr_dataset[0]

        random_index = random.randint(0, len(arr_dataset) - 1)
        
        return arr_dataset[random_index]

        
    def distance(self, arr_dataset, arr_tuple_of_medoids):
        ret_distance = {}
        for i in range(len(arr_dataset)):
            f1 = pow(arr_dataset[i]['F1'] - arr_tuple_of_medoids['F1'], EXPONENT)
            f2 = pow(arr_dataset[i]['F2'] - arr_tuple_of_medoids['F2'], EXPONENT)
            f3 = pow(arr_dataset[i]['F3'] - arr_tuple_of_medoids['F3'], EXPONENT)
            f4 = pow(arr_dataset[i]['F4'] - arr_tuple_of_medoids['F4'], EXPONENT)
            f5 = pow(arr_dataset[i]['F5'] - arr_tuple_of_medoids['F5'], EXPONENT)
            f6 = pow(arr_dataset[i]['F6'] - arr_tuple_of_medoids['F6'], EXPONENT)
            f7 = pow(arr_dataset[i]['F7'] - arr_tuple_of_medoids['F7'], EXPONENT)
            f8 = pow(arr_dataset[i]['F8'] - arr_tuple_of_medoids['F8'], EXPONENT)
            sumF = f1 + f2 + f3 + f4 + f5 + f6 + f7 + f8
            ret_distance.update({i: sumF})
        return ret_distance
    
    def variance(self, arr_distance):
        return sum(arr_distance.values()) / len(arr_distance)
    
        
    def create_two_cluster(self, arr_distance_c1, arr_distance_c2):
        ret = {'C1': [], 'C2': []}
        
        for key, val in arr_distance_c1.items():
            if val < arr_distance_c2[key]:
                ret['C1'].append(key)
            else:
                ret['C2'].append(key)
        
        return ret
    
    def bisecting_kmedoids_clustering(self):
        X = [self.datasets] # One set big cluster

        next_level = []
        V = X.copy() # One set big Cluster
        S = []
        arr_medoid_for_all_clusters = []


        while len(V) > 0:
            for val in V:
                if len(val) == 0:
                    break
                # print("VAL = ", val)
                arr_random_medoid_tuple_parent = self.random_medoid_tuple(val)
                parent_distance = self.distance(val, arr_random_medoid_tuple_parent)
                parent_variance = self.variance(parent_distance)

                # split menjadi dua cluster(C1, C2)
                arr_random_medoid_tuple_c1 = self.random_medoid_tuple(val)
                arr_random_medoid_tuple_c2 = self.random_medoid_tuple(val)
                arr_distance_c1 = self.distance(val, arr_random_medoid_tuple_c1)
                arr_distance_c2 = self.distance(val, arr_random_medoid_tuple_c2)
                variance_c1 = self.variance(arr_distance_c1)
                variance_c2 = self.variance(arr_distance_c2)
                two_cluster_c1_c2 = self.create_two_cluster(arr_distance_c1, arr_distance_c2)

                if max(variance_c1, variance_c2) < parent_variance:
                    temp = []
                    for vals in two_cluster_c1_c2:
                        for subval in two_cluster_c1_c2[vals]:
                            temp.append(val[subval])
                        next_level.append(temp)
                        temp = []
                else:
                    S.append(val)
                    arr_medoid_for_all_clusters.append(arr_random_medoid_tuple_parent)
                
            V = next_level
            next_level = []

        ret = {'medoids': arr_medoid_for_all_clusters, 'clusters': S}
        # print(ret['medoids'])
        # print("Cluster lenght: ", len(ret['clusters']))
        # print(ret['clusters'])
        self.clusters = ret
        return ret

    # create cluster name in datsets
    def clusters_naming(self):
        clusters =  self.clusters

        for i in range (len(clusters['clusters'])):
            for j in range (len(clusters['clusters'][i])):
                clusters['clusters'][i][j].update({"cluster": i})

        return clusters

In [5]:
# bisecting K-medoid with Leave One Out
# return data bisecting to csv

class ClusterGeneration:
    
    # leave one data
    def __init__(self, index):
        DatasetsClass = DatasetHelper()  #class call
        # return all datasets minus 1
        all_datasets = DatasetsClass.dataset_leave_one(index) 

        # running generate cluster
        self.bisecting_k_medoid_run(all_datasets)

        # check length cluster if less than 120 data
        clusters_len = pd.read_csv('clusters.csv')
        data_len = clusters_len.shape[0]
        while data_len < 119 :
            self.bisecting_k_medoid_run(all_datasets) # Generate cluster again

             # check again if length cluster if less than 119 data
            clusters_len = pd.read_csv('clusters.csv')
            data_len = clusters_len.shape[0]

    # calling bisecting k-medoid to generate cluster dataset
    def bisecting_k_medoid_run(self, all_datasets):


        # k_medoid class call
        k_medoid_class = BisectingKMedoid(all_datasets) 
        
        k_medoid_clustering = {}  
        k_medoid_clustering['clusters'] = []  # Initialize 'clusters' to an empty list
        while len(k_medoid_clustering['clusters']) < 2:  # repeat until there are more than 1 clusters
            k_medoid_clustering = k_medoid_class.bisecting_kmedoids_clustering() 

        named_clusters = k_medoid_class.clusters_naming() # clusters naming

        clusters_generated = []
        clusters_generated.append(named_clusters)

        # Save into csv
        datasets = named_clusters['clusters']

        # convert dataset train into one dimensional array
        data_dict = []
        for i in range(len(datasets)):
            for j in range(len(datasets[i])):
                data_dict.append(datasets[i][j])
                # print(datasets[i][j])

        # Construct DataFrame from dict
        df = pd.DataFrame.from_dict(data_dict) # data train
        df.to_csv('clusters.csv') # save to csv
        
        medoids = named_clusters['medoids']
        # Construct DataFrame from dict
        df_medoids = pd.DataFrame.from_dict(medoids) # data train
        df_medoids.to_csv('medoids.csv')


In [6]:
# TEST
# Runnning Cluster Generate minus one index
bisect = ClusterGeneration(119)

In [7]:
# # check length cluster
clusters_len = pd.read_csv('clusters.csv')
clusters_len.shape

(119, 13)

## 3. Support Vector Machine



In [8]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [9]:
class BisectingSupportVectorMachine:

    # svm to predicting cluster class
    def support_vector_machine(self, param_C, param_gamma, loo_index=1):

        df = pd.read_csv('clusters.csv')
  
        # split into x and y
        X = df.loc[:, ['F1', 'F2','F3','F4','F5','F6','F7','F8']] # data x
        y = df.loc[:, ['cluster']] # data y
        y = y.values.ravel() # convert into one dimensional array

        X_train, X_test, y_train, y_test = train_test_split(X,y ,test_size=0.25)
        unique_classes = np.unique(y_train)
        if len(unique_classes) <= 1: 
            # print(f"Only one class present in training data: {unique_classes}. Skipping SVM training.")
            ClusterGeneration(loo_index)
            X_train, X_test, y_train, y_test = train_test_split(X,y ,test_size=0.25)
            self.support_vector_machine(param_C, param_C, loo_index)
            

        # create a svm Classifier
        svm_classifier = SVC(kernel='rbf', C=param_C, gamma=param_gamma)

        # Train the model
        svm_classifier.fit(X_train, y_train)

        # Predict the response for test dataset
        y_pred = svm_classifier.predict(X_test)
        
        accuracy_score = metrics.accuracy_score(y_test, y_pred)

        # Predict cluster on data test
        df_test = pd.read_csv('datasetPF.csv')
        predicted_cluster = df_test.loc[df_test.project == loo_index, ['F1', 'F2','F3','F4','F5','F6','F7','F8']] # data x
        pred_cluster = svm_classifier.predict(predicted_cluster)[0]

        return accuracy_score, pred_cluster

In [10]:
# svm = BisectingSupportVectorMachine()

# accuracy_score, pred_cluster = svm.support_vector_machine(80.2, 2.1, loo_index=1)

# print(accuracy_score, pred_cluster)

## 4. Grey Wolf Optimizer

In [33]:
import sys, copy
import numpy as np

class GreyWolfOptimizer():
    def __init__(self, parameters, loo_index):
        self.parameters = parameters
        self.loo_index = loo_index
        self.fe = 0
        self.max_fe = 2 * 10000

        
    def obj_func(self, x):
        svm = BisectingSupportVectorMachine() 
        accuracy_score, _ = svm.support_vector_machine(x[0], x[1], self.loo_index)
        self.fe += 1
        return accuracy_score
    
    def initialpop(self):
        population = []
        for i in range(self.parameters['popsize']):
            wolf = [random.uniform(self.parameters['C'][0], self.parameters['C'][1]),
                    random.uniform(self.parameters['g'][0], self.parameters['g'][1])]
            population.append(wolf)
        return population 
        
    def isPositionOutOfRange(self, index, newPosition):
        if index == 0:  # Checking for 'C'
            if newPosition < self.parameters['C'][0]:
                return self.parameters['C'][0]
            elif newPosition > self.parameters['C'][1]:
                return self.parameters['C'][1]
        elif index == 1:  # Checking for 'g'
            if newPosition < self.parameters['g'][0]:
                return self.parameters['g'][0]
            elif newPosition > self.parameters['g'][1]:
                return self.parameters['g'][1]
        return newPosition
    
    def preyHunting(self, population, wolfType, C, A):
        for wolf in population:
            for i in range(len(wolfType)):
                D = abs(C * wolfType[i] - wolf[i])
                newPosition = wolfType[i] - A * D
                newPosition = self.isPositionOutOfRange(i, newPosition)
                wolf[i] = newPosition
        return population
    
    def runGWO(self):
        bestWolf = {
            'wolfPositions': None,
            'ObjectValue': 0
        }
        bestConvergences = []
        population = self.initialpop()
        
    
        for i in range(self.parameters['maxIter']):
            
            ob = np.array([self.obj_func(p) for p in population])
            sorted_indices = np.argsort(ob)[::-1]  # Sort indices of ob in descending order
            sorted_ob = ob[sorted_indices]
            best_ob = sorted_ob[:3]
            best_wolf = np.array(population)[sorted_indices][:3]
            
            
            alphawolf = best_wolf[0]
            betawolf = best_wolf[1]
            deltawolf = best_wolf[2]
            
            print('AlphaWolf : ', best_wolf[0])
            print('BetaWolf : ', betawolf)
            print('DeltaWolf : ', deltawolf)
            sys.exit()
            
            coefficient = 2
            a = coefficient - (coefficient * i) / self.parameters['maxIter']
            A = (coefficient * a) * random.uniform(0, 1) - a
            C = coefficient * random.uniform(0, 1)
            alphapopulation = self.preyHunting(population, alphawolf, C, A)
            bestConvergences.append(bestWolf['ObjectValue'])
            if best_ob[0] > bestWolf['ObjectValue']:
                bestWolf['wolfPositions'] = copy.deepcopy(alphawolf.tolist())
                bestWolf['ObjectValue'] = copy.deepcopy(best_ob[0])
            betapopulation = self.preyHunting(population, betawolf, C, A)
            deltapopulation = self.preyHunting(population, deltawolf, C, A)
            
            for i in range (self.parameters['popsize']):
                positions = []
                for j in range (2):
                    position = (alphapopulation[i][j]+betapopulation[i][j]+deltapopulation[i][j])/2
                    positions.append(position)
                population[i] = positions
        return bestWolf
       
    
            
            

In [34]:
parameters = {'maxIter': 25, 
              'popsize': 5,
              'C': [0.01, 100],
              'g': [0.01, 50]}
gwo = GreyWolfOptimizer(parameters, loo_index=1)
gwo.runGWO()

# # Load existing Excel file or create a new DataFrame
# excel_filename = 'gwo_results.xlsx'
# try:
#     df = pd.read_excel(excel_filename)
# except FileNotFoundError:
#     df = pd.DataFrame(columns=['maxIter', 'popsize', 'wolfPositions', 'ObjectValue'])

# # Lists to store results
# results = []

# # Run GWO multiple times
# for i in range(30):
#     hasil = gwo.runGWO()
#     results.append({
#         'maxIter': parameters['maxIter'],
#         'popsize': parameters['popsize'],
#         'wolfPositions': str(hasil['wolfPositions']),
#         'ObjectValue': hasil['ObjectValue']
#     })

# # Append new results to DataFrame
# df_new = pd.DataFrame(results)
# df = pd.concat([df, df_new], ignore_index=True)

# # Export DataFrame to Excel
# df.to_excel(excel_filename, index=False)

# print(f"Results appended to {excel_filename}")
    

AlphaWolf :  0.3333333333333333
BetaWolf :  [93.25056838 20.58916653]
DeltaWolf :  [85.00520199 42.02328814]


SystemExit: 

C:\Users\Gifan\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 5. Menghitung nilai Productivity Factor (PF)
- perhitungan pf hanya diambil dari tuple medoid
- pf = medoid[i] {actual effort/size ucp}

In [13]:
class ProductivityFactorCalc:

    def calculate_pf_medoid(self):
        df_medoids = pd.read_csv('medoids.csv')
        df_medoids = df_medoids.drop(df_medoids.columns[0], axis=1)
        
        # print(df_medoids)

        pf = []
        for ind in range (len(df_medoids)):
            actual_effort = df_medoids['actual_effort'][ind]
            size_ucp = df_medoids['size_ucp'][ind]
            pf.append(actual_effort/size_ucp)

        return pf
    
    def main(self):
        pf_medoid = self.calculate_pf_medoid()

        df_cluster = pd.read_csv('clusters.csv')
        df_cluster = df_cluster.drop(df_cluster.columns[0], axis=1)
        df_cluster

        df_medoids = pd.read_csv('medoids.csv')
        df_medoids = df_medoids.drop(df_medoids.columns[0], axis=1)
        df_medoids

        pf_arr = []
        for ind in range (len(df_medoids)):
            for i in range (len(pf_medoid)):
                df_cluster.loc[df_cluster["cluster"] == i, "pf"] = pf_medoid[i]

        return df_cluster
    

In [14]:
# # TEST
pf_calc = ProductivityFactorCalc()

pf_medoid = pf_calc.calculate_pf_medoid()
print(pf_medoid)

all_clusters = pf_calc.main()
all_clusters

[24.0, 32.0]


,project,actual_effort,size_ucp,F1,F2,F3,F4,F5,F6,F7,F8,cluster,pf
0,0,2124,118,2,2,2,2,2,3,4,0,0,24.0
1,1,1430,130,4,3,3,4,4,3,3,0,0,24.0
2,2,1445,85,3,3,3,3,3,4,3,0,0,24.0
3,3,4895,275,4,4,4,5,4,4,2,0,0,24.0
4,4,2420,110,3,2,2,3,3,3,4,0,0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,113,1734,102,5,2,2,4,5,3,3,5,1,32.0
115,114,2668,116,3,2,3,3,3,3,2,5,1,32.0
116,115,2832,118,3,3,3,3,3,3,4,5,1,32.0
117,116,4680,156,1,3,3,2,1,4,4,5,1,32.0


## 6. Estimasi Effort
- Estimasi untuk menentukan nilai 'effort' dari ucp
- Metode Regresi Linear
- Variabel Bebas : Size dan PF Terprediksi
- Variabel Terikat : effort terestimasi

In [15]:
from sklearn import linear_model


def estimated_effort_calc(all_clusters, loo_index, pf_pred):  
    X_var = all_clusters[['size_ucp', 'pf']]
    y_var = all_clusters['actual_effort']

    # linear regression
    regr = linear_model.LinearRegression()
    regr.fit(X_var.values, y_var) # fit model

    data_test = pd.read_csv('datasetPF.csv')
    size_test = data_test['size_ucp'][loo_index]

    #predict effort where 'size' and 'pf' is
    predicted_est_eff = regr.predict([[size_test, pf_pred]])

    return predicted_est_eff


## 7. Metrics
- Menghitung performa estimasi
- metrics: 
  1. Absolute error
  2. Mean Absolute Error
  3. Standarized accuracy
  4. Effect Size

In [16]:
import numpy as np
import sys

# Standardized Accuray and Effect Size
class SAES:
    sumSA = 0.0
    recurrSumAE = 0.0

    def __init__(self, MAEPi, actualEfforts, numRuns = 1000):
        self.MAEPi = MAEPi
        self.actualEfforts = actualEfforts
        self.numRuns = numRuns

    def getRandomGuessing(self):
        
        selectedEstimateds = []
        numActualEffort = len(self.actualEfforts)
        
        for i in range(numActualEffort):
            actualEffortsAfterDelete = np.delete(self.actualEfforts, i)
            selectedEstimated = np.random.choice(actualEffortsAfterDelete, 1)
            selectedEstimateds.append(selectedEstimated[0])
        
        return selectedEstimateds    
    
    def calcSAES(self):
        
        P0 = []
        numData = len(self.getRandomGuessing())
        
        for _ in range(self.numRuns):
            estimateds = self.getRandomGuessing()
            for j in range(numData):
                ae = abs(estimateds[j] - self.actualEfforts[j])           
                self.recurrSumAE += ae
                
            mae = self.recurrSumAE / numData
            self.sumSA += mae
            P0.append(mae)
            mae = 0.0
            
        MAEP0 = self.sumSA / self.numRuns
        SA = (1 - (self.MAEPi / MAEP0)) * 100
        ES = abs(self.MAEPi - MAEP0) / np.std(P0)
        return [SA, ES]

## Main Driver


In [17]:
def main(particles, iter): 
    # loo_index = 29
    absolute_err_arrray = []

    # call data absolute error
    data_act_eff = pd.read_csv('datasetPF.csv')
    data_act_eff = data_act_eff['actual_effort']
    
    for loo_index in range(120):
        print(loo_index)
        # Runnning Cluster Generate minus
        bisect = ClusterGeneration(loo_index)

        # Grey Wolf Optimizer
        parameters = {
            'C': [0.1, 1000],  # Range for C parameter in SVM
            'g': [0.1, 1],     # Range for gamma parameter in SVM
            'popsize': particles,   # Number of particles
            'maxIter': iter         # Maximum iterations
        }
        
        gwo = GreyWolfOptimizer(parameters, loo_index)
        best_wolf = gwo.runGWO()

        solution_hyperparameter = best_wolf['wolfPositions']
        fitness = best_wolf['ObjectValue']
        
        # print(solution_hyperparameter, fitness)
        # print("Pred Cluster :", pred_cluster)

        # SVM with optimized parameters
        svm = BisectingSupportVectorMachine()
        _, pred_cluster = svm.support_vector_machine(solution_hyperparameter[0], solution_hyperparameter[1], loo_index)

        # Calculate PF each medoids
        pf_calc = ProductivityFactorCalc()
        pf_pred = pf_calc.calculate_pf_medoid()[pred_cluster]

        # print(pf_pred)

        all_clusters = pf_calc.main()
        # all_clusters
        hasil = estimated_effort_calc(all_clusters, loo_index, pf_pred)

        # calculating absolute error
        absolute_err = abs(data_act_eff[loo_index] - hasil) 
        # print("Estimated Effort : ", hasil, "AE", absolute_err[0])

        absolute_err_arrray.append(absolute_err[0])

    # calculating Mean absolute error    
    mean_absolute_err = sum(absolute_err_arrray) / len(absolute_err_arrray)
    # print(mean_absolute_err)

    # calculate Standardized Accuracy and Effect Size
    saes = SAES(mean_absolute_err, data_act_eff)
    StdAcc, EffSize = saes.calcSAES()
    # print("SA, ES", StdAcc, EffSize)

    return mean_absolute_err, StdAcc, EffSize


In [18]:
main(5,5)

0
[[845.8023303067849, 0.9576126360297614], [479.37834613883274, 0.6676942310701375], [816.4980636181554, 0.3816973424640008], [579.8639427574419, 0.15388351859509666], [471.51642262018805, 0.21941009369048387]]


SystemExit: 

C:\Users\Gifan\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
result = []

for i in range (30):
    res = main(25, 25)

    result.append(res)
    print(i, res)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
0 (460.6375945231095, 99.93873095127579, 1.7335905238002212)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
1 (453.03329893500273, 99.93960543125847, 1.732714959742792)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
4

KeyboardInterrupt: 

In [ ]:
result_df = pd.DataFrame(result, columns=['mean_absolute_error', 'standarized_accuracy', 'effect_size'])
result_df.to_csv('part5iter20.csv')
result_df

,mean_absolute_error,standarized_accuracy,effect_size
0,460.637595,99.938731,1.733591
1,453.033299,99.939605,1.732715
2,460.537410,99.938591,1.733448
